# Import and Load

In [49]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from typing import Tuple, List
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
%pip install catboost
import catboost as cb
import os
import joblib

import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm # For progress bars
import random

Note: you may need to restart the kernel to use updated packages.


In [51]:
RANDOM_SEED = 69
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

In [52]:
# train_df = pd.read_csv('/content/drive/MyDrive/Education/University/Master/ML/tr_clean.csv')
# test_df = pd.read_csv('/content/drive/MyDrive/Education/University/Master/ML/te_clean.csv')

In [53]:
train_df = pd.read_csv('tr_clean.csv')
test_df = pd.read_csv('te_clean.csv')

# Data Prep.

## Inspection

In [54]:
# Get columns of type object
object_cols = train_df.select_dtypes(include=['object']).columns

# Print object columns
print("Columns of type object:")
print(object_cols)

# Drop object columns
train_df = train_df.drop(columns=object_cols)
test_df = test_df.drop(columns=object_cols)

print(train_df.info())
print(test_df.info())


Columns of type object:
Index(['description', 'zone'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4494 entries, 0 to 4493
Columns: 109 entries, price to sqm_x_high_efficiency
dtypes: float64(48), int64(61)
memory usage: 3.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2833 entries, 0 to 2832
Columns: 108 entries, square_meters to sqm_x_high_efficiency
dtypes: float64(47), int64(61)
memory usage: 2.3 MB
None


In [55]:
def plot_top_correlations(X: pd.DataFrame, 
                         y: pd.Series, 
                         n_features: int = 30,
                         figsize: tuple = (12, 10),
                         cmap: str = 'coolwarm',
                         mask_diagonal: bool = True):
    """
    Create and plot a correlation heatmap for the top n most correlated features.
    
    Parameters:
    -----------
    X : pandas DataFrame
        Feature matrix
    y : pandas Series
        Target variable
    n_features : int, default=30
        Number of top correlated features to include in the plot
    figsize : tuple, default=(12, 10)
        Figure size for the plot
    cmap : str, default='coolwarm'
        Color map for the heatmap
    mask_diagonal : bool, default=True
        Whether to mask the diagonal of the correlation matrix
    
    Returns:
    --------
    fig : matplotlib Figure
        The generated figure
    top_corr_df : pandas DataFrame
        DataFrame containing the correlation values for the top features
    """
    # Combine features and target
    data = pd.concat([X, y], axis=1)
    
    # Calculate correlation matrix
    corr_matrix = data.corr()
    
    # Get average absolute correlation for each feature
    avg_corr = corr_matrix.abs().mean().sort_values(ascending=False)
    
    # Select top n features (excluding the target if it's in the top)
    top_features = avg_corr.head(n_features + 1)  # +1 in case target is in top
    if y.name in top_features.index:
        top_features = avg_corr[avg_corr.index != y.name].head(n_features)
    else:
        top_features = top_features.head(n_features)
    
    # Add target variable to the list of features
    selected_features = list(top_features.index) + [y.name]
    
    # Create correlation matrix for selected features
    top_corr = corr_matrix.loc[selected_features, selected_features]
    
    # Create mask for diagonal if requested
    if mask_diagonal:
        mask = np.eye(len(selected_features), dtype=bool)
    else:
        mask = None
    
    # Create figure
    fig, ax = plt.subplots(figsize=figsize)
    
    # Create heatmap
    sns.heatmap(top_corr, 
                mask=mask,
                cmap=cmap,
                center=0,
                annot=True,
                fmt='.2f',
                square=True,
                linewidths=0.5,
                cbar_kws={'shrink': .5},
                ax=ax)
    
    # Rotate x-axis labels
    plt.xticks(rotation=70, ha='right')
    plt.yticks(rotation=0)
    
    # Add title
    plt.title(f'Correlation Heatmap: Top {n_features} Most Correlated Features\n',
              pad=20,
              fontsize=30)
    
    # Adjust layout
    plt.tight_layout()
    
    # Create summary of correlations with target
    target_corr = top_corr[y.name].sort_values(ascending=False)
    print("\nTop 20 Features Most Correlated with Target:")
    print(target_corr[target_corr.index != y.name].head(20))
    
    # Print features with highest average correlation
    print("\nFeatures with Highest Average Absolute Correlation:")
    print(avg_corr[avg_corr.index != y.name].head(10))
    
    return fig, top_corr

In [56]:
X_train = train_df.drop(columns=['price'])
y_train = train_df['price']

# fig, corr_matrix = plot_top_correlations(
#     X=X_train,
#     y=y_train,
#     n_features=50,
#     figsize=(20, 20),
#     cmap='coolwarm',
#     mask_diagonal=True
# )

## Transformations

### Normalise

In [57]:
def plot_feature_distributions(df):
    # looking to apply tranformations to narmalise
    plt.figure(figsize=(20, 100))

    # Calculate number of rows and columns for subplots
    n_features = len(train_df.columns)
    n_cols = 3
    n_rows = (n_features + n_cols - 1) // n_cols

    # Create subplots for each feature
    for idx, feature in enumerate(train_df.columns, 1):
        plt.subplot(n_rows, n_cols, idx)
        
        # Create histogram with KDE
        sns.histplot(data=train_df, x=feature, kde=True)
        plt.title(f'Distribution of {feature}')
        plt.xlabel(feature)
        plt.ylabel('Count')
        
    plt.suptitle('Feature distributions in Train df')
    plt.tight_layout()
    plt.show()

# plot_feature_distributions(train_df)

In [58]:
def log_transformations(df, variables, plot=False):
    """
    Apply np.log1p transformation to specified variables and plot before vs after distributions.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Input DataFrame containing the variables to transform
    variables : list
        List of column names to transform and plot
    
    Returns:
    --------
    transformed_df : pandas DataFrame
        DataFrame with added log-transformed columns
    """
    # Create a copy of the dataframe
    transformed_df = df.copy()
    
    # Set up the plotting area
    n_vars = len(variables)
    if plot:
        fig, axes = plt.subplots(n_vars, 2, figsize=(15, 4*n_vars))
        fig.suptitle('Original vs Log-Transformed Distributions', y=1.02, fontsize=16)
    
    # For each variable
    for idx, var in enumerate(variables):
        if var not in df.columns:
            continue
        # Create transformed column name
        log_var = f'log_{var}'
        
        # Apply log1p transformation
        transformed_df[log_var] = np.log1p(df[var])
        transformed_df = transformed_df.drop(columns=[var])

        if var == 'price':
            transformed_df = transformed_df.rename(columns={log_var: 'y'})
        
        if plot:
            # Plot original distribution
            sns.histplot(data=df, x=var, ax=axes[idx, 0], kde=True)
            axes[idx, 0].set_title(f'Original: {var}')
            axes[idx, 0].set_xlabel(var)
            
            # Plot transformed distribution
            sns.histplot(data=transformed_df, x=log_var, ax=axes[idx, 1], kde=True)
            axes[idx, 1].set_title(f'Log-Transformed: {var}')
            axes[idx, 1].set_xlabel(f'log1p({var})')
    
    if plot:
        plt.tight_layout()
    
    return transformed_df

# Variables to transform
variables_to_transform = [
    'square_meters',
    'sqm_x_zone_rank',
    'sqm_x_condition',
    'bedrooms_x_zone_rank'
]

train_df = log_transformations(train_df, variables_to_transform)
test_df = log_transformations(test_df, variables_to_transform)

### Scale

In [45]:
# Separate target variable before scaling
y_train = train_df['y']
X_train = train_df.drop('y', axis=1)

# Scale features
feature_scaler = MinMaxScaler()
X_train_scaled = pd.DataFrame(
    feature_scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

# Scale test features - ensure columns match training data
test_df_scaled = pd.DataFrame(
    feature_scaler.transform(test_df[X_train.columns]),
    columns=X_train.columns,
    index=test_df.index
)

# Modelling

## Tree-Based

### Setup

In [140]:
SCORING = 'neg_mean_absolute_error' # For RandomizedSearchCV/GridSearchCV

def evaluate_model(y_true, y_pred, model_name="Model"):
    """Calculates and prints regression metrics."""
    y_true_eval = np.array(y_true).copy() 
    y_pred_eval = np.array(y_pred).copy()

    mse = mean_squared_error(y_true_eval, y_pred_eval)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_eval, y_pred_eval)
    r2 = r2_score(y_true_eval, y_pred_eval)
    
    print(f"--- {model_name} Performance ---")
    print(f"MAE (Primary): {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("----------------------------- ulcers")
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

N_SPLITS = 5
cv_strategy = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)

In [146]:
X_train_data = train_df.drop(columns=['y'])
y_train_data = train_df['y']

X_test_data = test_df

X_train, X_dev, y_train, y_dev = train_test_split(
    X_train_data, 
    y_train_data, 
    test_size=0.09, 
    random_state=RANDOM_SEED
)

print(f"Original full train data shape: {X_train_data.shape}")
print(f"New X_train shape: {X_train.shape}, new y_train shape: {y_train.shape}")
print(f"X_dev shape: {X_dev.shape}, y_dev shape: {y_dev.shape}")
print("Data loaded, prepared, and split into train/dev sets.")

Original full train data shape: (4494, 108)
New X_train shape: (4089, 108), new y_train shape: (4089,)
X_dev shape: (405, 108), y_dev shape: (405,)
Data loaded, prepared, and split into train/dev sets.


### Baseline Models

In [149]:
baseline_models = {
    "RandomForest": RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1),
    "XGBoost": xgb.XGBRegressor(random_state=RANDOM_SEED, n_jobs=-1),
    "LightGBM": lgb.LGBMRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbosity=-1),
    "CatBoost": cb.CatBoostRegressor(random_state=RANDOM_SEED, verbose=0)
}

baseline_results = {}
do_baseline = False

if do_baseline:
    for name, model in baseline_models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        y_pred_dev = model.predict(X_dev) # Evaluate on the dev set
        baseline_results[name] = evaluate_model(y_dev, y_pred_dev, 
                                                model_name=f"Baseline {name}",
                                                target_transformer=target_scaler)

    print("\nBaseline Model Training Complete.")
    print("\nSummary of Baseline Results (on Dev Set):")
    print("-" * 50)
    for model_name, metrics in baseline_results.items():
        print(f"\n{model_name}:")
        print(f"  MSE:  {metrics['mse']:.4f}")
        print(f"  RMSE: {metrics['rmse']:.4f}")
        print(f"  MAE:  {metrics['mae']:.4f}")
        print(f"  R2:   {metrics['r2']:.4f}")
    print("-" * 50)

Training RandomForest...
--- Baseline RandomForest Performance ---
MSE: 0.0054
RMSE: 0.0737
MAE: 0.0513
R2 Score: 0.8113
-----------------------------
Training XGBoost...
--- Baseline XGBoost Performance ---
MSE: 0.0057
RMSE: 0.0756
MAE: 0.0531
R2 Score: 0.8014
-----------------------------
Training LightGBM...
--- Baseline LightGBM Performance ---
MSE: 0.0055
RMSE: 0.0740
MAE: 0.0517
R2 Score: 0.8096
-----------------------------
Training CatBoost...
--- Baseline CatBoost Performance ---
MSE: 0.0055
RMSE: 0.0742
MAE: 0.0518
R2 Score: 0.8088
-----------------------------

Baseline Model Training Complete.

Summary of Baseline Results (on Dev Set):
--------------------------------------------------

RandomForest:
  MSE:  0.0054
  RMSE: 0.0737
  MAE:  0.0513
  R2:   0.8113

XGBoost:
  MSE:  0.0057
  RMSE: 0.0756
  MAE:  0.0531
  R2:   0.8014

LightGBM:
  MSE:  0.0055
  RMSE: 0.0740
  MAE:  0.0517
  R2:   0.8096

CatBoost:
  MSE:  0.0055
  RMSE: 0.0742
  MAE:  0.0518
  R2:   0.8088
------

### Hyperparameter Optimisation

In [150]:
N_ITER_RANDOM_SEARCH = 50 # Number of parameter settings that are sampled for RandomizedSearchCV.

tuned_models = {}
best_params_all_models = {}

#### Random Forest

In [151]:
rf_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', 0.5, 0.7, 1.0],
    'bootstrap': [True, False]
}

rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1),
    param_distributions=rf_param_grid,
    n_iter=N_ITER_RANDOM_SEARCH,
    cv=cv_strategy,
    scoring=SCORING,
    random_state=RANDOM_SEED,
    n_jobs=-1, # Use all available cores
    verbose=1
)
rf_random_search.fit(X_train, y_train)
tuned_models["RandomForest"] = rf_random_search.best_estimator_
best_params_all_models["RandomForest"] = rf_random_search.best_params_
print(f"Best RandomForest Params: {rf_random_search.best_params_}")
print(f"Best RandomForest Score (CV {SCORING}, on transformed y): {rf_random_search.best_score_:.4f}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best RandomForest Params: {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.5, 'max_depth': 30, 'bootstrap': True}
Best RandomForest Score (CV neg_mean_squared_error): -0.0050


#### XGBoost

In [ ]:
xgb_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2]
}
xgb_random_search = RandomizedSearchCV(
    xgb.XGBRegressor(random_state=RANDOM_SEED, n_jobs=-1, objective='reg:absoluteerror'), # Explicitly set objective
    param_distributions=xgb_param_grid,
    n_iter=N_ITER_RANDOM_SEARCH,
    cv=cv_strategy,
    scoring=SCORING,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=1
)
xgb_random_search.fit(X_train, y_train)
tuned_models["XGBoost"] = xgb_random_search.best_estimator_
best_params_all_models["XGBoost"] = xgb_random_search.best_params_
print(f"Best XGBoost Params: {xgb_random_search.best_params_}")
print(f"Best XGBoost Score (CV {SCORING}, on transformed y): {xgb_random_search.best_score_:.4f}")

#### CatBoost

In [ ]:
cat_param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300, 500], # n_estimators for CatBoost
    'depth': [4, 6, 8, 10], # max_depth
    'l2_leaf_reg': [1, 3, 5, 7], # L2 regularization
    'rsm': [0.8, 0.9, 1.0],  # colsample_bylevel (analogous to colsample_bytree)
    'subsample': [0.8, 0.9, 1.0], # for CatBoost, this is only effective if bootstrap_type is 'Bernoulli' or 'Poisson'
    'min_data_in_leaf': [1, 10, 20] # min_child_samples
}
cat_random_search = RandomizedSearchCV(
    cb.CatBoostRegressor(random_state=RANDOM_SEED, verbose=0, allow_writing_files=False),
    param_distributions=cat_param_grid,
    n_iter=N_ITER_RANDOM_SEARCH,
    cv=cv_strategy,
    scoring=SCORING,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=1
)
cat_random_search.fit(X_train, y_train)
tuned_models["CatBoost"] = cat_random_search.best_estimator_
best_params_all_models["CatBoost"] = cat_random_search.best_params_
print(f"Best CatBoost Params: {cat_random_search.best_params_}")
print(f"Best CatBoost Score (CV {SCORING}, on transformed y): {cat_random_search.best_score_:.4f}")

#### Evaluate on Dev Set

In [ ]:
tuned_results_dev = {} # Renamed from tuned_results_test
for name, model in tuned_models.items():
    print(f"Evaluating Tuned {name}...")
    y_pred_dev = model.predict(X_dev) # Evaluate on the dev set
    tuned_results_dev[name] = evaluate_model(y_dev, y_pred_dev,
                                            model_name=f"Tuned {name}",
                                            target_transformer=target_scaler)

print("\nTuned Model Evaluation on Dev Set Complete.")
print("Tuned Model Dev Set Results:", tuned_results_dev)

### Ensemble Method

In [ ]:
estimators = []
for name, model in tuned_models.items():
    if model is not None: # Check if model was successfully tuned
        estimators.append((name.lower(), model)) # Naming convention for VotingRegressor

if estimators:
    voting_regressor = VotingRegressor(estimators=estimators, n_jobs=-1)

    print("Training Voting Regressor...")
    voting_regressor.fit(X_train, y_train)
    print("Evaluating Voting Regressor on Dev Set...")
    y_pred_ensemble_dev = voting_regressor.predict(X_dev) # Evaluate on the dev set
    ensemble_results_dev = evaluate_model(y_dev, y_pred_ensemble_dev, 
                                          model_name="Voting Ensemble",
                                          target_transformer=target_scaler)
    print("Voting Ensemble Dev Set Results:", ensemble_results_dev)
else:
    print("No tuned models available to create an ensemble.")

### Summary

In [ ]:
print("\n--- FINAL MODEL PERFORMANCE SUMMARY (Dev Set, metrics in original scale) ---")

print("\nBaseline Models (on Dev Set):")
for name, metrics in baseline_results.items():
    print(f"  {name}: MAE={metrics['mae']:.4f}, R2={metrics['r2']:.4f}")

print("\nTuned Individual Models (on Dev Set):")
for name, metrics in tuned_results_dev.items():
    print(f"  {name}: MAE={metrics['mae']:.4f}, R2={metrics['r2']:.4f}")

if 'ensemble_results_dev' in locals() and ensemble_results_dev:
    print("\nEnsemble Model (Voting Regressor on Dev Set):")
    print(f"  Voting Ensemble: MAE={ensemble_results_dev['mae']:.4f}, R2={ensemble_results_dev['r2']:.4f}")

In [ ]:
# Create models directory if it doesn't exist
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Created directory: {models_dir}")

# Save best individual models
for name, model in tuned_models.items():
    if model is not None:
        model_path = os.path.join(models_dir, f'best_{name.lower()}_model.joblib')
        joblib.dump(model, model_path)
        print(f"Saved {name} model to: {model_path}")

# Save ensemble model if it exists
if 'voting_regressor' in locals() and voting_regressor is not None:
    ensemble_path = os.path.join(models_dir, 'best_ensemble_model.joblib')
    joblib.dump(voting_regressor, ensemble_path)
    print(f"Saved Ensemble model to: {ensemble_path}")

# Save best parameters for each model
best_params_path = os.path.join(models_dir, 'best_parameters.json')
pd.Series(best_params_all_models).to_json(best_params_path)
print(f"Saved best parameters to: {best_params_path}")

# Save feature scaler if it exists (assuming it's defined in your data preprocessing)
if 'feature_scaler' in locals() and feature_scaler is not None:
    scaler_path = os.path.join(models_dir, 'feature_scaler.joblib')
    joblib.dump(feature_scaler, scaler_path)
    print(f"Saved feature scaler to: {scaler_path}")

# Save target transformer if it exists (assuming it's defined in your data preprocessing)
if 'target_scaler' in locals() and target_scaler is not None:
    target_scaler_path = os.path.join(models_dir, 'target_scaler.joblib')
    joblib.dump(target_scaler, target_scaler_path)
    print(f"Saved target scaler to: {target_scaler_path}")

print("\nAll models and parameters have been saved successfully.") 

### Further Steps

1. Analyze feature importances for the best model(s)
2. Perform GridSearchCV for more fine-grained tuning around the best parameters found by RandomizedSearchCV (optimizing for MAE)
3. Experiment with different ensemble weighting strategies, potentially based on MAE performance
4. If you obtain a separate, final TEST set (with labels), evaluate the chosen model on it for a final unbiased performance assessment
5. Analyze error patterns (e.g., residuals plot) for the best model on the dev set

## NN-based

### Setup

In [32]:
PRIMARY_METRIC = 'mae' # Mean Absolute Error
LOSS_FUNCTION_NAME = 'mean_absolute_error' # For consistency, actual loss is nn.L1Loss

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def evaluate_nn_model(y_true, y_pred, model_name="NN Model", target_transformer=None):
    """Calculates and prints regression metrics for NN.
    If target_transformer is provided, it will inverse transform y_true and y_pred 
    before calculating metrics to report them in the original scale.
    Assumes y_true and y_pred are in the log1p+scaled domain if transformer is provided.
    """
    y_true_eval_np = y_true.cpu().numpy() if isinstance(y_true, torch.Tensor) else np.array(y_true).copy()
    y_pred_eval_np = y_pred.cpu().numpy() if isinstance(y_pred, torch.Tensor) else np.array(y_pred).copy()

    if target_transformer is not None:
        if y_true_eval_np.ndim == 1:
            y_true_eval_np = y_true_eval_np.reshape(-1, 1)
        if y_pred_eval_np.ndim == 1:
            y_pred_eval_np = y_pred_eval_np.reshape(-1, 1)
        
        # Inverse MinMax scaling (applied on log1p transformed data)
        y_true_log_unscaled = target_transformer.inverse_transform(y_true_eval_np)
        y_pred_log_unscaled = target_transformer.inverse_transform(y_pred_eval_np)

        # Inverse log1p (expm1)
        y_true_eval_np = np.expm1(y_true_log_unscaled).flatten()
        y_pred_eval_np = np.expm1(y_pred_log_unscaled).flatten()
        # print(f"DEBUG: NN Reporting metrics in original scale for {model_name}")
    # else:
        # print(f"DEBUG: NN Reporting metrics in transformed scale for {model_name}")

    mae = mean_absolute_error(y_true_eval_np, y_pred_eval_np)
    mse = mean_squared_error(y_true_eval_np, y_pred_eval_np)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true_eval_np, y_pred_eval_np)
    
    print(f"--- {model_name} Performance (Original Scale if transformer provided) ---")
    print(f"MAE (Primary): {mae:.4f}")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("-----------------------------")
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

print(f"PyTorch Version: {torch.__version__}")

Using device: cpu
PyTorch Version: 2.2.2


In [30]:
X_train_data = train_df.drop(columns=['y'])
y_train_data = train_df['y']

X_test_data = test_df

X_train_df, X_dev_df, y_train_series, y_dev_series = train_test_split(
    X_train_data, 
    y_train_data, 
    test_size=0.09,
    random_state=RANDOM_SEED
)

print(f"Original full train data shape: {X_train_data.shape}")
print(f"New X_train shape: {X_train_df.shape}, new y_train shape: {y_train_series.shape}")
print(f"X_dev (validation) shape: {X_dev_df.shape}, y_dev (validation) shape: {y_dev_series.shape}")

X_train_np = X_train_df.values
X_dev_np = X_dev_df.values

y_train_np = y_train_series.values.reshape(-1, 1) 
y_dev_np = y_dev_series.values.reshape(-1, 1)

# Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
X_dev_tensor = torch.tensor(X_dev_np, dtype=torch.float32)
y_dev_tensor = torch.tensor(y_dev_np, dtype=torch.float32)

BATCH_SIZE = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False)

Original full train data shape: (4494, 108)
New X_train shape: (4089, 108), new y_train shape: (4089,)
X_dev (validation) shape: (405, 108), y_dev (validation) shape: (405,)


### Model Definition

In [24]:
class MLP(nn.Module):
    def __init__(self, input_dim, config, output_dim=1):
        super(MLP, self).__init__()
        layers_list = [] # Use layers_list instead of layers to avoid name conflict
        current_dim = input_dim
        
        num_hidden_layers = config['num_hidden_layers']
        units_per_layer = config['units_per_layer']
        activations_per_layer = config['activations_per_layer']
        dropout_rates = config['dropout_rates']

        for i in range(num_hidden_layers):
            layers_list.append(nn.Linear(current_dim, units_per_layer[i]))
            if activations_per_layer[i] == 'relu':
                layers_list.append(nn.ReLU())
            elif activations_per_layer[i] == 'tanh':
                layers_list.append(nn.Tanh())
            elif activations_per_layer[i] == 'elu':
                layers_list.append(nn.ELU())
            
            if dropout_rates[i] > 0:
                layers_list.append(nn.Dropout(dropout_rates[i]))
            current_dim = units_per_layer[i]
            
        layers_list.append(nn.Linear(current_dim, output_dim))
        self.network = nn.Sequential(*layers_list)

    def forward(self, x):
        return self.network(x)
    
def train_epoch(model, dataloader, criterion, optimizer, device, epoch_num, num_epochs, trial_num=None):
    model.train()
    total_loss = 0
    total_mae = 0
    
    desc = f"Epoch {epoch_num+1}/{num_epochs} [Train]"
    if trial_num is not None:
        desc = f"Trial {trial_num} - " + desc
    progress_bar = tqdm(dataloader, desc=desc, leave=False)
    
    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
        with torch.no_grad():
            mae = nn.L1Loss()(outputs, targets).item()
            total_mae += mae * inputs.size(0)
        progress_bar.set_postfix(loss=loss.item(), mae=mae)
        
    avg_loss = total_loss / len(dataloader.dataset)
    avg_mae = total_mae / len(dataloader.dataset)
    return avg_loss, avg_mae

def validate_epoch(model, dataloader, criterion, device, epoch_num, num_epochs, trial_num=None):
    model.eval()
    total_loss = 0
    total_mae = 0

    desc = f"Epoch {epoch_num+1}/{num_epochs} [Val]"
    if trial_num is not None:
        desc = f"Trial {trial_num} - " + desc    
    progress_bar = tqdm(dataloader, desc=desc, leave=False)
    with torch.no_grad():
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            mae = nn.L1Loss()(outputs, targets).item()
            total_mae += mae * inputs.size(0)
            progress_bar.set_postfix(loss=loss.item(), mae=mae)
            
    avg_loss = total_loss / len(dataloader.dataset)
    avg_mae = total_mae / len(dataloader.dataset)
    return avg_loss, avg_mae

### Baseline

In [31]:
input_dim_baseline = X_train_tensor.shape[1]
baseline_config = {
    'num_hidden_layers': 2,
    'units_per_layer': [64, 32],
    'activations_per_layer': ['relu', 'relu'],
    'dropout_rates': [0.0, 0.0],
    'optimizer_type': 'adam',
    'learning_rate': 0.001
}

baseline_nn_model_pt = MLP(input_dim_baseline, baseline_config).to(device)
optimizer_baseline = optim.Adam(baseline_nn_model_pt.parameters(), lr=baseline_config['learning_rate'])
criterion_baseline = nn.L1Loss()
scheduler_baseline = ReduceLROnPlateau(optimizer_baseline, mode='min', factor=0.2, patience=5)

EPOCHS_BASELINE = 100
PATIENCE_BASELINE = 15
best_val_mae_baseline = float('inf')
patience_counter_baseline = 0
best_baseline_model_state_dict = None

history_baseline_pt = {'loss': [], 'val_loss': [], PRIMARY_METRIC: [], f'val_{PRIMARY_METRIC}': []}

In [ ]:
for epoch in range(EPOCHS_BASELINE):
    train_loss, train_mae = train_epoch(baseline_nn_model_pt, train_loader, criterion_baseline, optimizer_baseline, device, epoch, EPOCHS_BASELINE)
    val_loss, val_mae = validate_epoch(baseline_nn_model_pt, dev_loader, criterion_baseline, device, epoch, EPOCHS_BASELINE)
    history_baseline_pt['loss'].append(train_loss); history_baseline_pt[PRIMARY_METRIC].append(train_mae)
    history_baseline_pt['val_loss'].append(val_loss); history_baseline_pt[f'val_{PRIMARY_METRIC}'].append(val_mae)
    print(f"Baseline Epoch {epoch+1}/{EPOCHS_BASELINE} => Train Loss: {train_loss:.4f}, Train MAE: {train_mae:.4f} | Val Loss: {val_loss:.4f}, Val MAE: {val_mae:.4f}")
    scheduler_baseline.step(val_mae)
    
    if val_mae < best_val_mae_baseline:
        best_val_mae_baseline = val_mae
        patience_counter_baseline = 0
        best_baseline_model_state_dict = copy.deepcopy(baseline_nn_model_pt.state_dict())
        # torch.save(best_baseline_model_state_dict, 'best_baseline_nn_model_pt.pth') # Save if desired
        print(f"  Baseline Val MAE improved to {val_mae:.4f}.")
    else:
        patience_counter_baseline += 1
        # print(f"  Baseline Val MAE did not improve. Patience: {patience_counter_baseline}/{PATIENCE_BASELINE}")
    if patience_counter_baseline >= PATIENCE_BASELINE: print("Baseline early stopping."); break

print("\nEvaluating Baseline PyTorch NN Model on Dev Set...")
if best_baseline_model_state_dict: baseline_nn_model_pt.load_state_dict(best_baseline_model_state_dict)
baseline_nn_model_pt.eval()
all_preds_baseline = []
all_true_baseline = [] # For collecting y_dev_tensor parts
with torch.no_grad():
    for inputs, targets_batch in dev_loader: # targets_batch will be from y_dev_tensor
        all_preds_baseline.append(baseline_nn_model_pt(inputs.to(device)).cpu())
        all_true_baseline.append(targets_batch.cpu()) 

y_pred_baseline_dev_pt = torch.cat(all_preds_baseline) # Already a tensor
y_true_baseline_dev_pt = torch.cat(all_true_baseline) # Already a tensor

baseline_nn_results_pt = evaluate_nn_model(y_dev_series.values, y_pred_baseline_dev_pt, model_name="Baseline PyTorch NN")

pd.DataFrame(history_baseline_pt).plot(figsize=(8, 5)); plt.title("Baseline PyTorch NN Model Training History")
plt.gca().set_ylim(0, np.max(history_baseline_pt[f'val_{PRIMARY_METRIC}'])*1.2 if history_baseline_pt[f'val_{PRIMARY_METRIC}'] else None); plt.grid(True); plt.show()

### Hyperparameter Optimisation

In [26]:
param_grid = {
    'num_hidden_layers': [1, 2, 3],
    'units_options': [[32], [64], [128], [256], [64, 32], [128, 64], [256, 128], [128, 64, 32], [256, 128, 64]],
    'activation_options': ['relu', 'tanh', 'elu'],
    'dropout_options': [0.0, 0.1, 0.2, 0.3],
    'learning_rate_options': [1e-4, 5e-4, 1e-3, 5e-3, 1e-2],
    'optimizer_options': ['adam', 'rmsprop'] # NAdam can be added if preferred
}

N_RANDOM_TRIALS = 10 # Number of random configurations to try. Increase for better search.
TUNER_EPOCHS = 30    # Max epochs for each trial during tuning. Early stopping is active.
TUNER_PATIENCE = 7   # Patience for early stopping during tuning trials.

all_trial_results = []
best_trial_mae = float('inf')
best_trial_config = None
best_trial_model_state_dict = None

In [27]:
for trial_idx in range(N_RANDOM_TRIALS):
    print(f"\n--- Starting Trial {trial_idx+1}/{N_RANDOM_TRIALS} ---")
    
    # Sample random configuration
    config = {}
    config['num_hidden_layers'] = random.choice(param_grid['num_hidden_layers'])
    
    # Filter units_options based on num_hidden_layers
    possible_units = [u for u in param_grid['units_options'] if len(u) == config['num_hidden_layers']]
    if not possible_units: # Fallback if no exact match (shouldn't happen with good grid design)
        config['units_per_layer'] = random.choice(param_grid['units_options'])[0:config['num_hidden_layers']]
    else:
        config['units_per_layer'] = random.choice(possible_units)
        
    config['activations_per_layer'] = [random.choice(param_grid['activation_options']) for _ in range(config['num_hidden_layers'])]
    config['dropout_rates'] = [random.choice(param_grid['dropout_options']) for _ in range(config['num_hidden_layers'])]
    config['learning_rate'] = random.choice(param_grid['learning_rate_options'])
    config['optimizer_type'] = random.choice(param_grid['optimizer_options'])

    print(f"Trial {trial_idx+1} Config: {config}")

    trial_model = MLP(input_dim_baseline, config).to(device)
    if config['optimizer_type'] == 'adam':
        trial_optimizer = optim.Adam(trial_model.parameters(), lr=config['learning_rate'])
    elif config['optimizer_type'] == 'rmsprop':
        trial_optimizer = optim.RMSprop(trial_model.parameters(), lr=config['learning_rate'])
    # Add other optimizers if in options
    
    trial_criterion = nn.L1Loss()
    trial_scheduler = ReduceLROnPlateau(trial_optimizer, mode='min', factor=0.2, patience=3, verbose=False)

    current_best_trial_epoch_mae = float('inf')
    current_trial_patience_counter = 0
    current_best_trial_epoch_state_dict = None

    for epoch in range(TUNER_EPOCHS):
        train_loss, train_mae = train_epoch(trial_model, train_loader, trial_criterion, trial_optimizer, device, epoch, TUNER_EPOCHS, trial_num=trial_idx+1)
        val_loss, val_mae = validate_epoch(trial_model, dev_loader, trial_criterion, device, epoch, TUNER_EPOCHS, trial_num=trial_idx+1)
        # No extensive history tracking for each trial epoch to save memory, just print
        print(f"  Trial {trial_idx+1} - Epoch {epoch+1}/{TUNER_EPOCHS} => Train MAE: {train_mae:.4f} | Val MAE: {val_mae:.4f}")
        trial_scheduler.step(val_mae)

        if val_mae < current_best_trial_epoch_mae:
            current_best_trial_epoch_mae = val_mae
            current_trial_patience_counter = 0
            current_best_trial_epoch_state_dict = copy.deepcopy(trial_model.state_dict())
        else:
            current_trial_patience_counter += 1
        
        if current_trial_patience_counter >= TUNER_PATIENCE:
            print(f"  Trial {trial_idx+1} early stopping at epoch {epoch+1}.")
            break
    
    trial_result = {'config': config, 'val_mae': current_best_trial_epoch_mae, 'state_dict': current_best_trial_epoch_state_dict}
    all_trial_results.append(trial_result)
    print(f"Trial {trial_idx+1} Best Val MAE: {current_best_trial_epoch_mae:.4f}")

    if current_best_trial_epoch_mae < best_trial_mae:
        best_trial_mae = current_best_trial_epoch_mae
        best_trial_config = config
        best_trial_model_state_dict = current_best_trial_epoch_state_dict
        print(f"*** New Best Overall Trial MAE: {best_trial_mae:.4f} ***")

print("\nHyperparameter Optimization (Randomized Search) Complete.")
if best_trial_config:
    print(f"Best Trial Configuration Found: {best_trial_config}")
    print(f"Best Trial Validation MAE: {best_trial_mae:.4f}")
else:
    print("No successful trials completed or no improvement found.")

SyntaxError: EOL while scanning string literal (1667008824.py, line 66)

### Final Train

In [ ]:
if best_trial_config:
    print("\nBuilding and Training Final Best PyTorch NN Model from Search...")
    final_best_model_pt = MLP(input_dim_baseline, best_trial_config).to(device)
    if best_trial_config['optimizer_type'] == 'adam':
        final_optimizer = optim.Adam(final_best_model_pt.parameters(), lr=best_trial_config['learning_rate'])
    elif best_trial_config['optimizer_type'] == 'rmsprop':
        final_optimizer = optim.RMSprop(final_best_model_pt.parameters(), lr=best_trial_config['learning_rate'])

    final_criterion = nn.L1Loss()
    # Use longer patience for final model training
    final_scheduler = ReduceLROnPlateau(final_optimizer, mode='min', factor=0.2, patience=7, verbose=True) 

    FINAL_MODEL_EPOCHS = 150 # Max epochs for final model
    FINAL_MODEL_PATIENCE = 20 # Longer patience for final model
    
    current_best_final_model_val_mae = float('inf')
    final_model_patience_counter = 0
    # Load state from best trial if it makes sense, or train from scratch
    # For simplicity here, we train from scratch using best config. Can also load best_trial_model_state_dict.
    # if best_trial_model_state_dict: final_best_model_pt.load_state_dict(best_trial_model_state_dict)
    
    history_final_best_pt = {'loss': [], 'val_loss': [], PRIMARY_METRIC: [], f'val_{PRIMARY_METRIC}': []}

    for epoch in range(FINAL_MODEL_EPOCHS):
        train_loss, train_mae = train_epoch(final_best_model_pt, train_loader, final_criterion, final_optimizer, device, epoch, FINAL_MODEL_EPOCHS)
        val_loss, val_mae = validate_epoch(final_best_model_pt, dev_loader, final_criterion, device, epoch, FINAL_MODEL_EPOCHS)
        history_final_best_pt['loss'].append(train_loss); history_final_best_pt[PRIMARY_METRIC].append(train_mae)
        history_final_best_pt['val_loss'].append(val_loss); history_final_best_pt[f'val_{PRIMARY_METRIC}'].append(val_mae)
        print(f"Final Model Epoch {epoch+1}/{FINAL_MODEL_EPOCHS} => Train MAE: {train_mae:.4f} | Val MAE: {val_mae:.4f}")
        final_scheduler.step(val_mae)
        if val_mae < current_best_final_model_val_mae:
            current_best_final_model_val_mae = val_mae
            final_model_patience_counter = 0
            torch.save(final_best_model_pt.state_dict(), 'final_best_nn_model_pt.pth')
            print(f"  Final Model Val MAE improved to {val_mae:.4f}. Saving model.")
        else:
            final_model_patience_counter += 1
        if final_model_patience_counter >= FINAL_MODEL_PATIENCE: print("Final model early stopping."); break

    print("\nEvaluating Final Best PyTorch NN Model on Dev Set...")
    final_best_model_pt.load_state_dict(torch.load('final_best_nn_model_pt.pth')) # Load best saved state
    final_best_model_pt.eval()
    all_preds_final_best = []
    all_true_final_best = []
    with torch.no_grad():
        for inputs, targets_batch in dev_loader: 
            all_preds_final_best.append(final_best_model_pt(inputs.to(device)).cpu())
            all_true_final_best.append(targets_batch.cpu()) # Store the corresponding true values
    
    y_pred_final_best_dev_pt = torch.cat(all_preds_final_best) # Already a tensor
    y_true_final_best_dev_pt = torch.cat(all_true_final_best) # Already a tensor
    
    final_best_nn_results_pt = evaluate_nn_model(y_dev_series.values, y_pred_final_best_dev_pt, model_name="Final Best PyTorch NN")

    pd.DataFrame(history_final_best_pt).plot(figsize=(8, 5)); plt.title("Final Best PyTorch NN Model Training History (MAE on Transformed Target)")
    plt.gca().set_ylim(0, np.max(history_final_best_pt[f'val_{PRIMARY_METRIC}'])*1.2 if history_final_best_pt[f'val_{PRIMARY_METRIC}'] else None); plt.grid(True); plt.show()
else:
    print("Skipping final model training as no best trial configuration was found.")
    final_best_nn_results_pt = None # Ensure variable exists

### Evaluation

In [ ]:
print("\nBaseline PyTorch NN Model (on Dev Set):")
if 'baseline_nn_results_pt' in locals() and baseline_nn_results_pt:
    print(f"  MAE: {baseline_nn_results_pt['mae']:.4f}, MSE: {baseline_nn_results_pt['mse']:.4f}, R2: {baseline_nn_results_pt['r2']:.4f}")
else:
    print("  Baseline model results not available.")

print("\nFinal Best PyTorch NN Model from Randomized Search (on Dev Set):")
if final_best_nn_results_pt: # Check if it was set
    print(f"  MAE: {final_best_nn_results_pt['mae']:.4f}, MSE: {final_best_nn_results_pt['mse']:.4f}, R2: {final_best_nn_results_pt['r2']:.4f}")
else:
    print("  Final best model results not available (e.g., search yielded no improvements or was skipped).")


In [ ]:
# Create models directory if it doesn't exist
models_dir = 'models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Created directory: {models_dir}")

# Save baseline model if it exists
if 'baseline_nn_model_pt' in locals() and baseline_nn_model_pt is not None:
    baseline_path = os.path.join(models_dir, 'baseline_nn_model.pt')
    torch.save({
        'model_state_dict': baseline_nn_model_pt.state_dict(),
        'config': baseline_config,
        'performance': baseline_nn_results_pt if 'baseline_nn_results_pt' in locals() else None
    }, baseline_path)
    print(f"Saved baseline model to: {baseline_path}")

# Save final best model if it exists
if 'final_best_model_pt' in locals() and final_best_model_pt is not None:
    final_model_path = os.path.join(models_dir, 'final_best_nn_model.pt')
    torch.save({
        'model_state_dict': final_best_model_pt.state_dict(),
        'config': best_trial_config,
        'performance': final_best_nn_results_pt if 'final_best_nn_results_pt' in locals() else None
    }, final_model_path)
    print(f"Saved final best model to: {final_model_path}")

# Save hyperparameter search results
if 'all_trial_results' in locals() and all_trial_results:
    # Convert trial results to serializable format
    serializable_trials = []
    for trial in all_trial_results:
        serializable_trial = {
            'config': trial['config'],
            'val_mae': float(trial['val_mae'])  # Convert to Python float
        }
        serializable_trials.append(serializable_trial)
    
    trials_path = os.path.join(models_dir, 'hyperparameter_trials.json')
    with open(trials_path, 'w') as f:
        json.dump(serializable_trials, f, indent=2)
    print(f"Saved hyperparameter trials to: {trials_path}")

# Save best trial configuration if it exists
if 'best_trial_config' in locals() and best_trial_config is not None:
    best_config_path = os.path.join(models_dir, 'best_nn_config.json')
    with open(best_config_path, 'w') as f:
        json.dump(best_trial_config, f, indent=2)
    print(f"Saved best configuration to: {best_config_path}")

# Save feature scaler
if 'feature_scaler' in locals() and feature_scaler is not None:
    scaler_path = os.path.join(models_dir, 'feature_scaler.joblib')
    joblib.dump(feature_scaler, scaler_path)
    print(f"Saved feature scaler to: {scaler_path}")

# Save target transformer
if 'target_transformer_pt' in locals() and target_transformer_pt is not None:
    target_transformer_path = os.path.join(models_dir, 'target_transformer.joblib')
    joblib.dump(target_transformer_pt, target_transformer_path)
    print(f"Saved target transformer to: {target_transformer_path}")

print("\nAll models and parameters have been saved successfully.")

### Further steps

1. For more robust hyperparameter optimization in PyTorch, use dedicated libraries like Optuna or Ray Tune.
   - The randomized search here is a simplified simulation.

2. After finding a promising configuration, consider a more focused 'GridSearch-like' manual tuning around those best parameters.

3. Retrain the absolute best model on the *entire* X_full_train_df and y_full_train_series for a production model.
   - Remember to scale X_full_train_df appropriately
   - Evaluate on a true unseen TEST set if available

4. Experiment with batch_size as another hyperparameter to tune.

5. If categorical features are significant, consider using nn.Embedding layers.

6. Analyze error patterns (e.g., residuals plot) for the best model on the dev set.